### Model Training

In [3]:
import pandas as pd

In [4]:
df=pd.read_csv('data/gemstone.csv')
df.head()

,Unnamed: 0,carat,cut,color,clarity,depth,table,x,y,z,price
0,1,0.30,Ideal,E,SI1,62.1,58.0,4.27,4.29,2.66,499
1,2,0.33,Premium,G,IF,60.8,58.0,4.42,4.46,2.70,984
2,3,0.90,Very Good,E,VVS2,62.2,60.0,6.04,6.12,3.78,6289
3,4,0.42,Ideal,F,VS1,61.6,56.0,4.82,4.80,2.96,1082
4,5,0.31,Ideal,F,VVS1,60.4,59.0,4.35,4.43,2.65,779


In [7]:
df.drop(labels=['Unnamed: 0'],axis=1,inplace=True)

In [8]:
df.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,0.30,Ideal,E,SI1,62.1,58.0,4.27,4.29,2.66,499
1,0.33,Premium,G,IF,60.8,58.0,4.42,4.46,2.70,984
2,0.90,Very Good,E,VVS2,62.2,60.0,6.04,6.12,3.78,6289
3,0.42,Ideal,F,VS1,61.6,56.0,4.82,4.80,2.96,1082
4,0.31,Ideal,F,VVS1,60.4,59.0,4.35,4.43,2.65,779


In [10]:
## Splitting the data into dependent and independent variables
X = df.drop(labels=['price'],axis=1)
Y= df[['price']]

In [12]:
#Divide into categorical and numerical columns 
categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

In [14]:
##Custom Ranking for the ordinal categorical variables
cut_categories=['Fair','Good','Very Good','Premium','Ideal']
color_categories=['D','E','F','G','H','I','J']
clarity_categories=['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [17]:
from sklearn.impute import SimpleImputer ## For handling missing values
from sklearn.preprocessing import StandardScaler ## To perform standardization on data
from sklearn.preprocessing import OrdinalEncoder ## To perform Ordinal Encoder
##pipelines 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [16]:
X.isna().sum()

carat        0
cut          0
color        0
clarity      0
depth      697
table        0
x            0
y            0
z            0
dtype: int64

In [20]:
## Numerical Pipeline
num_pipeline =Pipeline(
    steps=
    [('imputer',SimpleImputer(strategy='median')),
     ('scaler',StandardScaler())
    ]
)

## Categorical Pipeline
cat_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
    ('scaler',StandardScaler())
    ]
)

preprocessor = ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)
])

In [27]:
##Train Test Split
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.30,random_state=42)

In [28]:
X_train.head()

,carat,cut,color,clarity,depth,table,x,y,z
19040,1.17,Good,D,SI2,60.4,65.0,6.81,6.77,4.10
396,1.20,Ideal,F,VVS1,61.1,55.0,6.86,6.89,4.20
15626,0.31,Very Good,I,VVS2,61.6,59.0,4.31,4.33,2.66
16597,2.19,Ideal,I,SI2,62.5,56.0,8.31,8.24,5.18
5811,0.30,Ideal,H,VS2,62.1,57.0,4.27,4.30,2.66


In [29]:
X_test.head()

,carat,cut,color,clarity,depth,table,x,y,z
9922,1.10,Very Good,F,SI1,60.5,60.0,6.65,6.67,4.03
25556,0.41,Good,E,SI2,63.5,55.0,4.69,4.73,2.99
22421,0.25,Premium,H,VVS1,59.8,59.0,4.07,4.09,2.44
18128,1.52,Ideal,H,VVS2,62.8,56.0,7.32,7.30,4.59
14105,0.42,Ideal,D,VVS2,62.1,55.0,4.78,4.81,2.98


In [30]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [31]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,0.787964,-0.966850,3.365840,0.964420,0.934381,0.813401,-1.717499,-1.523957,-1.247448
1,0.851153,-0.461986,-1.095845,1.008892,1.041947,0.957195,0.981578,-0.354480,1.791324
2,-1.023443,-0.101369,0.688829,-1.259163,-1.252793,-1.257232,-0.817807,1.399736,1.183569
3,2.936379,0.547743,-0.649677,2.298570,2.252064,2.366376,0.981578,1.399736,-1.247448
4,-1.044506,0.259249,-0.203508,-1.294740,-1.279684,-1.257232,0.981578,0.814997,-0.031940


In [32]:
X_test.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,0.640524,-0.894727,1.134997,0.822111,0.844743,0.712745,-0.817807,-0.354480,-0.639694
1,-0.812815,1.268977,-1.095845,-0.921178,-0.894240,-0.782712,-1.717499,-0.939218,-1.247448
2,-1.149821,-1.399591,0.688829,-1.472627,-1.467924,-1.573579,0.081886,0.814997,1.791324
3,1.525165,0.764113,-0.649677,1.418031,1.409464,1.517991,0.981578,0.814997,1.183569
4,-0.791752,0.259249,-1.095845,-0.841129,-0.822529,-0.797091,0.981578,-1.523957,1.183569


In [33]:
X_train.isna().sum()

num_pipeline__carat      0
num_pipeline__depth      0
num_pipeline__table      0
num_pipeline__x          0
num_pipeline__y          0
num_pipeline__z          0
cat_pipeline__cut        0
cat_pipeline__color      0
cat_pipeline__clarity    0
dtype: int64

In [34]:
X_test.isna().sum()

num_pipeline__carat      0
num_pipeline__depth      0
num_pipeline__table      0
num_pipeline__x          0
num_pipeline__y          0
num_pipeline__z          0
cat_pipeline__cut        0
cat_pipeline__color      0
cat_pipeline__clarity    0
dtype: int64

In [36]:
## Model Training
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [37]:
regression = LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [38]:
regression.coef_

array([[ 5134.94904902,   -67.41049312,   -63.44838633, -2608.5353019 ,
         1988.31634422,  -361.0516354 ,   138.46524479,  -572.15521544,
          817.73546177]])

In [39]:
regression.intercept_

array([3918.29561348])

In [40]:
import numpy as np
def evaluate_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mean_squared_error(true,predicted))
    r2_square=r2_score(true,predicted)
    return mae,mse,rmse,r2_square

In [42]:
#Train Multiple models
models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'ElasticNet':ElasticNet()
}
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #make predictions
    y_pred=model.predict(X_test)

    mae,mse,rmse,r2_square=evaluate_model(y_test,y_pred)
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print('RMSE :',rmse)
    print('MAE :',mae)
    print('R2 score:',r2_square*100)

    r2_list.append(r2_square)
    print('='*35)
    print('\n')

LinearRegression
Model Training Performance
RMSE : 1587.1167009084093
MAE : 835.886245456109
R2 score: 84.78189531683753




c:\Saravanesh Personal\INeuron\FSDS_Bootcamp\california_houseprice\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.581e+08, tolerance: 3.029e+07
  model = cd_fast.enet_coordinate_descent(


Lasso
Model Training Performance
RMSE : 1367.9530690614838
MAE : 830.6779148344891
R2 score: 88.69461803291462


Ridge
Model Training Performance
RMSE : 1561.7320006856874
MAE : 835.4727156232129
R2 score: 85.26480584830516


ElasticNet
Model Training Performance
RMSE : 1704.2038945761697
MAE : 1091.7710931670827
R2 score: 82.45368382630795


